In [ ]:
import requests
import pandas as pd
import time
import datetime
import os

# **DATA COLLECTION AND CLEANING**

In [ ]:
from google.colab import drive

# Mount Google Drive to Colab
drive.mount('/content/drive')

# Set the path to the directory where to save the CSV file
data_dir = "https://drive.google.com/drive/folders/1KlYSAQHsIcjDvKS4MZ-hy352_J3LFCYW?usp=share_link"

# Change the working directory to Google Drive directory
os.chdir('/content/drive/MyDrive')

# Verify that I am in the correct directory
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


# DATA COLLECTION: Zillow API

I will be getting data for recently sold properties, specifically for three different types of properties: Houses, TownHomes, and Condos. Zillow only allows to get data at most for 20 pages of information, with each page including 40 properties (max of 800 for each property type). To maximize the amount of data, I run three separate API calls for each type of property to get the max of 2400. I will then combine all three different property types to get a final list of recently sold properties.

In [ ]:
#Get a list of all the houses that were sold recently
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":"san francisco, ca","status_type":"RecentlySold","home_type":"Houses"}

headers = {
	"X-RapidAPI-Key": "c071463b35msh4722d8d40e4b0d3p1613bajsn7d170ee782bf",
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

all_results = []

for i in range(1, 21):
    querystring["page"] = i
    response = requests.request("GET", url, headers=headers, params=querystring)
    all_results.extend(response.json()["props"])
    i += 1

    time.sleep(0.5)


In [ ]:
#Turn the results into a dataframe
df_recently_sold_Houses = pd.json_normalize(all_results)

#Zillow only shows 20 pages of recent data so we have 400 rows (20 pages*20 properties per page)
df_recently_sold_Houses.shape

(800, 26)

In [ ]:
#Get a list of Townhomes that were sold recently
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":"san francisco, ca","status_type":"RecentlySold","home_type":"Townhomes"}

headers = {
	"X-RapidAPI-Key": "c071463b35msh4722d8d40e4b0d3p1613bajsn7d170ee782bf",
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

all_results = []

for i in range(1, 21):
    querystring["page"] = i
    response = requests.request("GET", url, headers=headers, params=querystring)
    all_results.extend(response.json()["props"])
    i += 1

    time.sleep(0.5)

In [ ]:
#Turn the results into a dataframe
df_recently_sold_Townhomes = pd.json_normalize(all_results)

In [ ]:
#Get a list of condos/apartments that were sold recently
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":"san francisco, ca","status_type":"RecentlySold","home_type":"Apartments_Condos_Co-ops"}

headers = {
	"X-RapidAPI-Key": "c071463b35msh4722d8d40e4b0d3p1613bajsn7d170ee782bf",
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

all_results = []

for i in range(1, 21):
    querystring["page"] = i
    response = requests.request("GET", url, headers=headers, params=querystring)
    all_results.extend(response.json()["props"])
    i += 1

    time.sleep(0.5)


In [ ]:
#Turn the results into a dataframe
df_recently_sold_Apartments_Condos = pd.json_normalize(all_results)

In [ ]:
#Combine the data for Houses+Townhomes+Condos
df_listings_Houses_Townhomes = pd.concat([df_recently_sold_Houses,df_recently_sold_Townhomes ], axis=0, ignore_index=True)
df_listings_Houses_Townhomes.shape

(1586, 26)

In [ ]:
#Combine the data from Houses and Townhomes to Condos to get a list of all property types
df_listings_All = pd.concat([df_listings_Houses_Townhomes,df_recently_sold_Apartments_Condos], axis=0, ignore_index=True)
df_listings_All.shape

(2386, 27)

In [ ]:
df_listings_All.columns

Index(['dateSold', 'propertyType', 'lotAreaValue', 'address', 'zestimate',
       'imgSrc', 'price', 'bedrooms', 'contingentListingType', 'longitude',
       'latitude', 'listingStatus', 'zpid', 'rentZestimate', 'daysOnZillow',
       'bathrooms', 'livingArea', 'country', 'currency', 'lotAreaUnit',
       'hasImage', 'variableData.text', 'variableData.type',
       'listingSubType.is_FSBA', 'unit', 'listingSubType.is_openHouse',
       'listingSubType.is_bankOwned'],
      dtype='object')

# DATA CLEANING

*   Deleteing columns that contain data that is not relevant
*   Deleting rows that contain data that is not relevant (i.e. rows where the property is Under_Contract and the sale has not yet been fully finalized)
*   Cleaning up the formatting for some of the rows (such as the date)


In [ ]:
#Find which rows and columns needs to be removed
df_listings_All['contingentListingType'].value_counts()
#contingentListingType - 2 are under_contract, remove these rows

df_listings_All["listingStatus"].value_counts()
#listingStatus - 4 are pending, remove these rows

df_listings_All["variableData.type"].value_counts()
#variableData.type - OPEN_HOUSE and ACCEPTS_APPLICATIONS, remove these rows

df_listings_All["variableData.type"].value_counts()
#variableData.type - OPEN_HOUSE and ACCEPTS_APPLICATIONS, remove these rows

df_listings_All["currency"].unique()
#all currency is dollar, so can remove this column

df_listings_All['listingSubType.is_openHouse'].value_counts()
#it's all NaN so can drop this column, same for listingSubType.is_bankOwned, listingSubType.is_FSBA

True    3
Name: listingSubType.is_openHouse, dtype: int64

In [ ]:
#Function to remove the rows
def remove_rows(df):
    # Remove rows where 'contingentListingType' is 'UNDER_CONTRACT'
    df = df[df['contingentListingType'] != 'UNDER_CONTRACT']

    # Remove rows where 'listingStatus' is 'PENDING'
    df = df[df['listingStatus'] != 'PENDING']

    # Remove rows where 'variableData.type' is 'OPEN_HOUSE' or 'ACCEPTS APPLICATIONS
    df = df[(df['variableData.type'] != 'OPEN_HOUSE') & (df['variableData.type'] != 'ACCEPTS_APPLICATIONS')]

    # Reset the index of the resulting DataFrame
    df.reset_index(drop=True, inplace=True)

    return df

df_listings_All = remove_rows(df_listings_All)

In [ ]:
#remove these columns: contingentListingType, ListingStatus, currency, variableData.type, variableData.text, listingSubType.is_FSBA, listingSubType.is_openHouse, listingSubType.is_bankOwned
df_listings_All.drop(['contingentListingType', 'listingStatus', 'currency', 'variableData.type', 'variableData.text', 'listingSubType.is_FSBA', 'listingSubType.is_openHouse', 'listingSubType.is_bankOwned'], axis=1, inplace=True)

In [ ]:
import datetime

def unix_timestamp_to_datetime(timestamp):
    if pd.isnull(timestamp):
        return None
    else:
        timestamp_sec = int(timestamp) / 1000
        dt = datetime.datetime.fromtimestamp(timestamp_sec)
        return dt

df_listings_All['dateSold'] = df_listings_All['dateSold'].apply(unix_timestamp_to_datetime)

In [ ]:
# Convert the 'dateSold' column to month/year format
df_listings_All['dateSold'] = pd.to_datetime(df_listings_All['dateSold'])

# Create a new column with the desired format
df_listings_All['month_year'] = df_listings_All['dateSold'].apply(lambda x: '{}/{}'.format(x.month, x.year))

In [ ]:
# Save the DataFrame to a CSV file
df_listings_All.to_csv(data_dir + "listings_data.csv", index=False)

In [ ]:
#Convert to csv and download
df_listings_All.to_csv('listings_All.csv', index=False)
from google.colab import files
files.download('listings_All.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **TEST DATA**

In [ ]:
#Get a list of all the houses that are avaialable for sell
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":"san francisco, ca","status_type":"ForSale","home_type":"Houses"}

headers = {
	"X-RapidAPI-Key": "c071463b35msh4722d8d40e4b0d3p1613bajsn7d170ee782bf",
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

all_results = []

for i in range(1, 21):
    querystring["page"] = i
    response = requests.request("GET", url, headers=headers, params=querystring)
    all_results.extend(response.json()["props"])
    i += 1

    time.sleep(0.5)

In [ ]:
#Turn the results into a dataframe
df_listing_Houses = pd.json_normalize(all_results)

#Zillow only shows 20 pages of recent data so we have 400 rows (20 pages*20 properties per page)
df_listing_Houses.shape

(784, 33)

In [ ]:
#Get a list of Townhomes that were sold recently
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":"san francisco, ca","status_type":"ForSale","home_type":"Townhomes"}

headers = {
	"X-RapidAPI-Key": "c071463b35msh4722d8d40e4b0d3p1613bajsn7d170ee782bf",
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

all_results = []

for i in range(1, 21):
    querystring["page"] = i
    response = requests.request("GET", url, headers=headers, params=querystring)
    all_results.extend(response.json()["props"])
    i += 1

    time.sleep(0.5)

In [ ]:
#Turn the results into a dataframe
df_listing_Townhomes = pd.json_normalize(all_results)

In [ ]:
#Get a list of condos/apartments that were sold recently
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":"san francisco, ca","status_type":"ForSale","home_type":"Apartments_Condos_Co-ops"}

headers = {
	"X-RapidAPI-Key": "c071463b35msh4722d8d40e4b0d3p1613bajsn7d170ee782bf",
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

all_results = []

for i in range(1, 21):
    querystring["page"] = i
    response = requests.request("GET", url, headers=headers, params=querystring)
    all_results.extend(response.json()["props"])
    i += 1

    time.sleep(0.5)

In [ ]:
#Turn the results into a dataframe
df_listing_Apartments_Condos = pd.json_normalize(all_results)

In [ ]:
#Combine the data for Houses+Townhomes+Condos
df_listings_for_sale_Houses_Townhomes = pd.concat([df_listing_Houses,df_listing_Townhomes], axis=0, ignore_index=True)
df_listings_for_sale_Houses_Townhomes.shape

(1004, 33)

In [ ]:
#Combine the data from Houses and Townhomes to Condos to get a list of all property types
df_listings_for_sale_All = pd.concat([df_listings_for_sale_Houses_Townhomes,df_listing_Apartments_Condos], axis=0, ignore_index=True)
df_listings_for_sale_All.shape

(1804, 33)

In [ ]:
df_listings_for_sale_All.to_csv('listings_for_sale_All.csv', index=False)
from google.colab import files
files.download('listings_for_sale_All.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>